In [3]:
import json
import requests
import sys 
sys.path.append("/scratch/izar/kapps/DEX-Cyclic-Arbitrage/")
from helper import *

In [5]:
def load_cycles(type_="raw",to_json=False):
    if type_== "raw":
        data_path = "/scratch/izar/kapps/DEX-Cyclic-Arbitrage/data/cycles_in_Uniswap.json"
    if type_== "filtered":
        data_path = "/scratch/izar/kapps/DEX-Cyclic-Arbitrage/data/filtered_cycles.json"
    data = []
    for line in open(data_path):
        if to_json:
            data.append(json.loads(line))
        else :
            data.append(line)
    return data

filtered_cycles = load_cycles("filtered")

cycle_paths = []
for cycle in filtered_cycles:
    cycle = json.loads(cycle)
    
    block_n  = max([x["blockNumber"] for x in cycle["receipt"]["logs"]])
    block_n  = int(block_n,16) 
    cycle_id = cycle["cycle_id"]
    path     = cycle["path"]
    
    cycle_paths.append({"block_n":block_n,"cycle_id":cycle_id,"path":path})

In [57]:
def create_query(k,token1,token2,block_n="12"):
    return ("""{
  ethereum(network: ethereum) {
    dexTrades(
      options: {limit: """+str(k)+""", asc: "timeInterval.minute"}
      exchangeName: {is: "Uniswap"}
      baseCurrency: {is: \""""+token1+"""\"}
      quoteCurrency: {is: \""""+token2+"""\"}
      hei}
    ) {
      timeInterval {
        minute(count: 5)
      }
      baseCurrency {
        symbol
        address
      }
      baseAmount
      quoteCurrency {
        symbol
        address
      }
      gasPrice
      gasValue
      quoteAmount
      trades: count
      quotePrice
      maximum_price: quotePrice(calculate: maximum)
      minimum_price: quotePrice(calculate: minimum)
      open_price: minimum(of: block, get: quote_price)
      close_price: maximum(of: block, get: quote_price)
    }
  }
}""")

In [58]:
k = 1000

uniswap_raw_data_path = "/scratch/izar/kapps/DEX-Cyclic-Arbitrage/data/uniswap_raw_data.json.bz2"
f_out = gzip.open(uniswap_raw_data_path,"a")
n_query = 0
for cycle_path in cycle_paths:
    block_n = cycle_path["block_n"]
    cycle_id = cycle_path["cycle_id"]
    path = cycle_path["path"]
    for i in range(0,len(path),2):
        if n_query>0 and n_query%10==0:
            time.sleep(60)
        
        pair = (path[i:i+2])
        fetched_data = {}
        fetched_data["pair"] = pair
        fetched_data["cycle_id"] = cycle_id
        query = create_query(k,token1=pair[0]
                                     ,token2=pair[1],
                                      block_n=block_n)
        if n_query%2==0:
            fetched_data["data"] = run_query(query,API_KEY1)
        else:
            fetched_data["data"] = run_query(query,API_KEY2)
            
        n_query+=1
        f_out.write(json.dumps(fetched_data)+"\n")
        print(f"{n_query}",end="\r")
        
    break
f_out.close()
print("DONE")

DONE


In [42]:
import time 
time.sleep(5)

In [45]:
str(str(block_n))

'10100398'

In [98]:
uniswap_raw_data_path = "/scratch/izar/kapps/DEX-Cyclic-Arbitrage/data/uniswap_raw_data_0_10.json.gz"
import gzip
data = []
for line in  gzip.open(uniswap_raw_data_path,"rt"):
    data.append(json.loads(line))